In [ ]:
# Premonitor: alert_manager.py
# This module handles the sending of all notifications and alerts.
# For the MVP, it focuses on sending a detailed email alert.
# In future versions, this will be expanded to include Twilio SMS/Voice calls.

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from datetime import datetime
import os
import threading

# Import our custom project configuration
import config

def send_alert_in_background(alert_type, details, image_path=None):
    """
    Starts the alert sending process in a separate thread to prevent
    blocking the main monitoring loop.
    """
    alert_thread = threading.Thread(
        target=send_email_alert,
        args=(alert_type, details, image_path)
    )
    alert_thread.start()
    if config.DEBUG_MODE:
        print(f"ALERT_MANAGER: Dispatched '{alert_type}' alert to run in background.")

def send_email_alert(alert_type, details, image_path=None):
    """
    Sends an email alert to the recipient defined in the config file.
    This function is designed to be run in a separate thread.

    Args:
        alert_type (str): A short description of the alert (e.g., "Thermal Anomaly Detected").
        details (str): A more detailed message explaining the alert.
        image_path (str, optional): The path to an image file to attach (e.g., a Grad-CAM heatmap).
    """
    # Check if email credentials are configured in the environment
    if not config.EMAIL_SENDER_ADDRESS or not config.EMAIL_SENDER_PASSWORD:
        print("ALERT_MANAGER: Email credentials not set in environment variables. Cannot send alert.")
        return False

    recipient = config.DEFAULT_EMAIL_RECIPIENT # This can be overridden by device-specific config later
    print(f"ALERT_MANAGER: Preparing to send '{alert_type}' email to {recipient}...")

    # --- Construct the Email ---
    msg = MIMEMultipart()
    msg['Subject'] = f"[Premonitor Alert] - {alert_type}"
    msg['From'] = config.EMAIL_SENDER_ADDRESS
    msg['To'] = recipient

    # Email Body
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    body = f"""
    Premonitor has detected a potential issue.

    Alert Type: {alert_type}
    Timestamp: {timestamp}

    Details:
    {details}

    Please review the attached data if available.
    """
    msg.attach(MIMEText(body, 'plain'))

    # --- Attach Image if Provided ---
    if image_path and os.path.exists(image_path):
        try:
            with open(image_path, 'rb') as f:
                img_data = f.read()
            image = MIMEImage(img_data, name=os.path.basename(image_path))
            msg.attach(image)
            if config.DEBUG_MODE:
                print(f"ALERT_MANAGER: Attached image '{os.path.basename(image_path)}'.")
        except Exception as e:
            print(f"ALERT_MANAGER: Error attaching image: {e}")

    # --- Send the Email ---
    try:
        if config.DEBUG_MODE:
            print(f"ALERT_MANAGER: Connecting to SMTP server {config.SMTP_SERVER}:{config.SMTP_PORT}...")
        # Use a secure context manager for the connection
        with smtplib.SMTP(config.SMTP_SERVER, config.SMTP_PORT) as server:
            server.starttls() # Secure the connection
            server.login(config.EMAIL_SENDER_ADDRESS, config.EMAIL_SENDER_PASSWORD)
            server.send_message(msg)
            print("ALERT_MANAGER: Email alert sent successfully!")
            return True
    except Exception as e:
        print(f"ALERT_MANAGER: Failed to send email. Error: {e}")
        return False

# This block allows you to test the email functionality directly.
if __name__ == '__main__':
    print("--- Running Alert Manager Test ---")

    # Check if credentials are set before running the test
    if not config.EMAIL_SENDER_ADDRESS or not config.EMAIL_SENDER_PASSWORD:
        print("\nERROR: Please set your email credentials as environment variables to run this test.")
        print("Example (Linux/macOS):")
        print("export EMAIL_SENDER_ADDRESS='your_email@gmail.com'")
        print("export EMAIL_SENDER_PASSWORD='your_google_app_password'")
    else:
        # Create a dummy image file for testing the attachment.
        dummy_image_file = "test_alert_image.png"
        try:
            from PIL import Image
            img = Image.new('RGB', (100, 50), color = 'red')
            img.save(dummy_image_file)
            print(f"Created dummy image: {dummy_image_file}")
        except ImportError:
            print("Pillow library not installed (pip install Pillow). Skipping dummy image creation.")
            dummy_image_file = None
        except Exception as e:
            print(f"Error creating dummy image: {e}")
            dummy_image_file = None

        # Test the function
        print("\nSending test alert in the background...")
        send_alert_in_background(
            alert_type="System Test",
            details="This is a test of the Premonitor alerting system. If you received this, the email configuration is working correctly.",
            image_path=dummy_image_file
        )
        print("Test alert dispatched. Please check the recipient's inbox in a few moments.")

        # Clean up the dummy image after a short delay to allow the thread to access it
        time.sleep(5)
        if dummy_image_file and os.path.exists(dummy_image_file):
            os.remove(dummy_image_file)